Partie 1

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv
import folium
import folium.plugins

%matplotlib inline


In [ ]:
transport_nodes = pd.read_csv('transport-nodes.csv')
print(transport_nodes.to_string())

In [ ]:
transport_relationships = pd.read_csv('transport-relationships.csv')
print(transport_relationships.to_string())

In [ ]:

relationships = transport_relationships[['src', 'dst']]
population_list = []
G = nx.Graph()
G = nx.from_pandas_edgelist(relationships, 'src', 'dst')

for n in G.nodes:
    i = 0
    while(i != len(transport_nodes.id)):
        if n == transport_nodes.id[i]:
            population_list.append(transport_nodes.population[i]/550)
        i += 1

plt.figure(figsize=(15, 8))
nx.draw(G, with_labels=True, 
edge_color='red', 
width=3, node_color='yellow', 
node_size=population_list)
dict(G.nodes.data())


In [ ]:
def ajouterAttribut(myGraphe, dfnode, nameAttr, Index):
    '''ajouter un attribut donné à tous les noeuds d'un graphe'''
    nodes = dfnode[[Index, nameAttr]]
    node_attr = nodes.set_index(Index).to_dict('index')
    nx.set_node_attributes(myGraphe, node_attr)

In [ ]:
ajouterAttribut(G, transport_nodes, 'latitude', 'id')
ajouterAttribut(G, transport_nodes, 'longitude', 'id')
ajouterAttribut(G, transport_nodes, 'population', 'id')

In [ ]:
dict(G.nodes.data())

In [ ]:
for i in G.nodes:
    basemap = folium.Map(location=[G.nodes[i]['latitude'], 
    G.nodes[i]['longitude']], 
    zoom_start=5, 
    tiles="Stamen Toner")

In [ ]:
basemap

In [ ]:
def marker(g):
    '''Ajout des marqueurs pour tous les noeuds du reseau'''
    for i in g.nodes:
        nodes = dict(G.nodes.data())
        latitude = nodes[i]['latitude']
        longitude = nodes[i]['longitude']
        folium.Marker(
            location=[latitude, longitude],
            icon=folium.plugins.BeautifyIcon(background_color='black', 
            text_color='white', icon='info'),
            popup=i,
            tooltip="Cliquez pour plus d'informations"
        ).add_to(basemap)
    return basemap

In [ ]:
marker(G)

In [ ]:
def construirePointsImage(myGraphe):
    '''Representation des coordonnées d'un point et d'un de ses voisins'''
    points = []
    nodes = dict(G.nodes.data())
    for i in myGraphe.nodes:
        cordonnees_node = []
        node_latitude = nodes[i]['latitude']
        node_longitude = nodes[i]['longitude']
        cordonnees_node.append(node_latitude)
        cordonnees_node.append(node_longitude)
        
        for neighbor in myGraphe.neighbors(i):
            cordonnees_neighbor = []
            neighbor_latitude = nodes[neighbor]['latitude']
            neighbor_longitude = nodes[neighbor]['longitude']
            cordonnees_neighbor.append(neighbor_latitude)
            cordonnees_neighbor.append(neighbor_longitude)
            
            list_neighbors = []
            list_neighbors.append(cordonnees_node)
            list_neighbors.append(cordonnees_neighbor)

            points.append(list_neighbors)

    return points

In [ ]:
coordonneesvoisins = construirePointsImage(G)
print(coordonneesvoisins)

In [ ]:
def visualiserFolium(myGraphe, points, locationpardefaut=[52.3791890, 4.899431], 
tiles='Stamen Toner', explored=None):
    '''Retourne une carte avec ses noeuds et arcs sous forme de ligne'''
    basemap = folium.Map(location=locationpardefaut, zoom_start=5, 
    tiles=tiles, explored=explored)
    points = construirePointsImage(myGraphe)
    folium.PolyLine(points, color="yellow", weight=3).add_to(basemap)
    for i in myGraphe.nodes:
        nodes = dict(G.nodes.data())
        latitude = nodes[i]['latitude']
        longitude = nodes[i]['longitude']
        folium.Marker(
            location=[latitude, longitude],
            icon=folium.plugins.BeautifyIcon(background_color='black', 
            text_color='white', icon='info'),
            popup=i,
            tooltip="Cliquez pour plus d'informations"
        ).add_to(basemap)
    return basemap

In [ ]:
visualiserFolium(G, coordonneesvoisins)

Partie 2

In [ ]:
class Noeud:
	'''Manipuler un noeud avec toutes les méthodes nécessaires'''
	def __init__(self, name):
		self.name = name
		self.attributs = {}
		self.listeNomVoisin = []

	def setAttribut(self, key, values):
		'''Ajout d'un attribut'''
		self.attributs.update({key:values})

	def getAttribut(self, key):
		'''Retourne un attribut donné en paramètre s'il existe'''
		if key not in self.attributs:
			return "Clé inexistante"
		else:
			return self.attributs[key]

	def getName(self):
		'''Retourne le nom du noeud'''
		return self.name

	def egal(self, noeud):
		'''Renvoie True si les noeuds ont le meme nom'''
		if(self.name == noeud.name):
			return True
		else:
			return False
	
	def getCoutMin(self):
		'''Renvoi le cout minimal d'une liste de noeuds'''
		noeuds = transport_relationships[['src', 'dst', 'cost']].to_dict('index')
		cout = 500
		for noeud in noeuds:
			if(noeuds[noeud]['src'] == self.name and 
			noeuds[noeud]['cost'] < cout):
				cout = noeuds[noeud]['cost']
			if(noeuds[noeud]['dst'] == self.name and 
			noeuds[noeud]['cost'] < cout):
				cout = noeuds[noeud]['cost']
		if cout != 500:
			return "Le coût minimal du noeud \"" + self.name +"\" est de : " + str(cout)
		else:
			return "Le noeud \"" + self.name +"\" est introuvble"

	def getCout(self, noeud):
		'''Retourne le cout d'une liste de noeuds'''
		nodes = transport_relationships[['src', 'dst', 'cost']].to_dict('index')
		cost = 0
		for node in nodes:
			if((nodes[node]['src'] == self.name and 
			nodes[node]['dst'] == noeud.name) or 
			(nodes[node]['src'] == noeud.name and 
			nodes[node]['dst'] == self.name)):
				cost = nodes[node]['cost']
		
		if cost != 0:
			return cost
		else:
			return "L'un des noeuds est introuvable ou il n'existe pas de relation entre \"" + self.name +"\" et \"" + noeud.name +"\""



In [ ]:
class Graphe:
	'''Réalisation de graphe en utilisant les concepts de l'orienté Objet'''
	def __init__(self):
		self.noeuds = []
		self.arcs = {}
	
	def creerNoeuds(self, fichiernoeuds):
		'''
			Retourne une liste de noms de noeuds
			Retourne une exception si le bon fichier n'a pas été ouvert
		'''
		try:
			transport_nodes = csv.reader(open(fichiernoeuds, 'r', newline=''))
			for node in transport_nodes:
				if node[0] != 'id':
					noeud = Noeud(node[0])
					noeud.setAttribut(key='coordonnes', 
					values=[float(node[1]),
					float(node[2])])
					noeud.setAttribut(key='population', 
					values=node[3])
					self.noeuds.append(noeud)
					self.arcs.update({noeud.getName() : []})
		except:
			print("Impossible d'ouvrir le fichier \'"+ fichiernoeuds + "\'." )
			return

		if(fichiernoeuds != 'transport-nodes.csv'):
			print("Vous n'avez pas ouvert le bon fichier.")
			return

		liste_noms_noeuds = []
		for noeud in self.noeuds:
			liste_noms_noeuds.append(noeud.getName())
		print(liste_noms_noeuds)

	def creerArc(self, fichierarcs):
		'''
			Retourne une liste d'arcs
			Retourne une exception si le bon fichier n'a pas été ouvert
		'''	
		try:
			transport_relationships = csv.reader(open(fichierarcs, 'r',
			newline=''))
		except:
			print("Impossible d'ouvrir le fichier\'"+ fichierarcs + "\'.")
			return   
		if(fichierarcs != 'transport-relationships.csv'):
			print("Vous n'avez pas ouvert le bon fichier.")
			return
		liste_arcs = []
		for relation in transport_relationships:
			if relation[0] != 'src':
				for noeud in self.noeuds:
					if(relation[0] == noeud.name):
						noeud.listeNomVoisin.append(relation[1])
					if(relation[1] == noeud.name):
						noeud.listeNomVoisin.append(relation[0])
				liste_arcs.append({"src":relation[0], "dst":relation[1],
				"relationship":relation[2], "cost":relation[3]})	
		for node in self.noeuds:
			voisinList = []
			for voisin in node.listeNomVoisin:
				voisinList.append((voisin, node.getCout(Noeud(voisin))))
			self.arcs.update({node.getName() : voisinList})
		return liste_arcs
    
	def getNoeud(self, name):
		'''
			Recherche et retourne le noeud ayant pour nom le nom passé
			en paramètre
			Retourne "noeud introuvable" si le nom ne correspond pas
		'''	
		for noeud in self.noeuds:
			if(name == noeud.getName()):
				return noeud
			if(name != noeud.getName() and noeud.getName() == self.noeuds[-1].
			getName()):
				return "Noeud introuvable"
	
	def getVoisins(self, noeud):
		'''Retourne la liste des voisins du noeud passé en paramètre'''
		list_voisins = []
		for node in self.noeuds:
			if noeud.egal(node):
				for voisin in node.listeNomVoisin:
					list_voisins.append(self.getNoeud(voisin))
		return list_voisins

	def getCoordonnesVoisins(self, noeud):
		'''
			Retourne une liste de coordonnes des noeuds voisins
			du noeud passé en parametre
		'''	
		coordonneesvoisins = []
		for voisin in self.getVoisins(noeud):
			coordonneesvoisins.append([self.getNoeud(noeud.name).
			getAttribut('coordonnes'), voisin.getAttribut('coordonnes')])
		return coordonneesvoisins
	
	def getListeCoordonnes(self, listeNoeuds):
		'''Retourne une liste de coordonnes de noeuds'''
		listeCoordonnes = []
		for noeud in listeNoeuds:
			listeCoordonnes.append(self.getNoeud(noeud).
			getAttribut('coordonnes'))
		return listeCoordonnes

	def visualiserFolium(self, locationpardefaut=[52.3791890, 4.899431], 
		tiles='Stamen Toner', explored=None):
		'''Retourne une carte'''
		basemap = folium.Map(location=locationpardefaut, 
		tiles=tiles, 
		zoom_start=7)
		points = []
		tooltip = "Cliquez pour plus d'informations"
		for noeud in self.noeuds:
			points.extend(self.getCoordonnesVoisins(noeud))
			folium.Marker(
				location=noeud.getAttribut("coordonnes"),
				popup=noeud.getName() + "\n\nCe noeud n'a pas été exploré",
				icon=folium.plugins.BeautifyIcon(background_color='black', 
				text_color='white', icon='info'),
				tooltip=tooltip
            ).add_to(basemap)
		if explored is not None:
				i = 0
				for coordonnes in self.getListeCoordonnes(explored):
					folium.Marker(
                        location=coordonnes,
                        popup=explored[i] + "\n\nCe noeud a été exploré",
                        icon=folium.plugins.BeautifyIcon(background_color='green', 
						text_color='white', icon='info'),
						tooltip=tooltip
                    ).add_to(basemap)
					i += 1
		folium.PolyLine(locations=points, color="red", weight=3).add_to(basemap)
		return basemap


In [ ]:
G = Graphe()
G.creerNoeuds('transport-nodes.csv')

Partie 3

In [ ]:
G.creerArc('transport-relationships.csv')

In [ ]:
G.visualiserFolium()

In [ ]:
class Pile():
    '''Manipulation de la structure de donnée Pile'''
    def __init__(self):
        self.elements = []

    def push(self, noeud):
        '''Insère un objet en première position de la liste'''
        self.elements.insert(0, noeud)
    
    def contains_noeud(self, name):
        '''Retourne True si un noeud est dans la pile'''
        for element in self.elements:
            if(element.egal(Noeud(name)) == True):
                return True
        return False        
        
    def empty(self):
        '''Retourne true si la pile est vide'''
        if len(self.elements) == 0:
            return True
        return False
    
    def remove(self):
        '''
            Retourne et supprime l'element en tete de pile
            Retourne une exception si la pile est vide
        '''
        try:
            return self.elements.pop(0)
        except:
            print("Pile vide")

# Test des structures de données Pile et File
f = Pile()
f.push("Mamadou")
f.push("Mansour")
f.push("Dame")
f.push("Khady")
print(f.elements)
f.remove()
print(f.elements)

In [ ]:
class File(Pile):
    '''Manipulation de la structure de données File en héritant de la classe Pile'''

    def remove(self):
        '''
            Retourne et supprime le premier élément de la File
            Retourne une exception si la File est vide
        '''
        try:
            return self.elements.pop()
        except:
            print("Suppression impossible")
            return

p = File()
p.push("Mamadou")
p.push("Mansour")
p.push("Dame")
p.push("Khady")
print(p.elements)
p.remove()
print(p.elements)

Partie 4

In [ ]:
def parcoursDFS(graphe : Graphe, noeud_source : Noeud, noeud_destination : Noeud):
    '''Permet de parcourir un graphe et de le visualiser'''
    explored = File()
    frontiere = Pile()
    for voisin in graphe.getVoisins(noeud_source):
        frontiere.push(voisin)
    explored.push(noeud_source.name)
    if(noeud_destination.egal(noeud_source) == True):
        return explored.elements
    while(frontiere.empty() != True):
        remove = frontiere.remove()
        explored.push(remove.name)
        if(noeud_destination.egal(remove) == True):
            return explored.elements
        voisins = graphe.getVoisins(remove)
        for voisin in voisins:
            if (voisin.name not in explored.elements and frontiere.
            contains_noeud(voisin.name) == False):
                frontiere.push(voisin)
    print('Pas de solution')
    return

In [ ]:
g = Graphe()
g.creerNoeuds('transport-nodes.csv')
g.creerArc('transport-relationships.csv')
g.visualiserFolium(explored=parcoursDFS(g, g.noeuds[0], g.noeuds[1]))
